In [3]:
import os
import numpy as np
import get_stock_data as gd
import stock_plot as sp
import pickle
import matplotlib.pyplot as plt
import get_market_data as gmd
data = gmd.Rate_return_compile()
with open("sp500tickers.pickle", "rb") as f:
    tickers = pickle.load(f)
# there might be tickers that the web can not reach
tickers = [t for t in tickers if os.path.exists('stock_dfs/{}.csv'.format(t))]
data = data.fillna(method='ffill')
data = data.fillna(0)

                 MMM       ABT      ABBV       ACN      ATVI       AYI  \
Date                                                                     
2016-01-04       NaN       NaN       NaN       NaN       NaN       NaN   
2016-01-05 -0.004340  0.000233  0.004183 -0.005178  0.012924 -0.012533   
2016-01-06  0.020555  0.008459 -0.000174  0.001958  0.009513  0.027094   
2016-01-07  0.024970  0.024555  0.002972  0.030254  0.014337  0.027252   
2016-01-08  0.003417  0.021392  0.028032  0.009776  0.015682  0.038797   
2016-01-11  0.000214 -0.001473  0.032851 -0.010380 -0.013808  0.044346   
2016-01-12 -0.002840 -0.017607 -0.017505 -0.017622 -0.024515 -0.013294   
2016-01-13  0.015427  0.029295  0.071512  0.017630  0.064831  0.047803   
2016-01-14 -0.017425 -0.019951 -0.061950 -0.025717 -0.015254 -0.035894   
2016-01-15  0.017954  0.013814 -0.048483  0.026292  0.014036  0.019403   

                ADBE       AMD       AAP       AES    ...         WYNN  \
Date                                 

In [4]:
# constuct X, construct Y
stock = 'GE'
lag = 7
nx = lag*data.shape[1]
X = np.array([[]]*nx)
Y = []
for i in range(data.shape[0]-lag):
    x = data.iloc[i:i+lag]
    xval = x.values
    xval = xval.reshape((xval.shape[1]*lag,1))
    X = np.column_stack((X,xval))
    if data[stock][i+lag]>=0:
        Y.append(1)
    else:
        Y.append(0)
m = len(Y)
Y = np.array([Y])
Y = Y.reshape((1,m))

In [5]:
# construct train_set_x, train_set_y   construct test_set_x, test_set_y 
trainportion = 0.8
check = int(trainportion*m)
X_train = X[:,:check]
Y_train = Y[0,:check].reshape(1,check)
X_test = X[:,check:]
Y_test = Y[0,check:].reshape(1,m-check)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(3514, 317)
(1, 317)
(3514, 80)
(1, 80)


In [6]:
# logistic regression
import logistic_regression as lr
model = lr.model(X_train, Y_train, X_test, Y_test, num_iterations=3000, learning_rate=0.0001, print_cost=False)

train accuracy: 46.37223974763407 %
test accuracy: 52.5 %


In [7]:
# 2 level neural network
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100, 3), random_state=1)
a = clf.fit(X_train.T, Y_train.flatten()) 
Y_predict = clf.predict(X_test.T)
Y_test = Y_test.flatten()
match = [1 for i in range(len(Y_test)) if Y_predict[i] == Y_test[i]]
sum(match)/len(Y_test)

0.525

In [8]:
print(tickers)

['MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AMD', 'AAP', 'AES', 'AET', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANDV', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'ADM', 'ARNC', 'AJG', 'AIZ', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BCR', 'BAX', 'BBT', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO', 'BF.B', 'CHRW', 'CA', 'COG', 'CPB', 'COF', 'CAH', 'CBOE', 'KMX', 'CCL', 'CAT', 'CBG', 'CBS', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CHK', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'COH', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'GLW', 'COST', 'COTY', 'CCI', 'CSRA', 'CSX', 'CMI', '